# Minesweeper Puzzle

The player is presented with a grid of squares. Some randomly selected squares, unknown to the player, contain mines. Other squares may display numbers indicating the count of mines present in the immediate neighborhood of the squares. Find all mines. 

See https://de.wikipedia.org/wiki/Minesweeper or go to http://minesweeperonline.com/ to play

Imports

In [1]:
from ortools.constraint_solver import pywrapcp
from itertools import product
import numpy as np

Pretty-print square minesweeper board

In [2]:
def pretty_print(mines, labels):
    for i in range(len(labels)):
        for j in range(len(labels)):
            if mines[i][j].Value() == 1:
                print("[💣]\t", end='')
            elif labels[i][j] == "":
                print("[  ]\t", end='')
            else:
                print("[ {}]\t".format(labels[i][j]), end='')
        print("\n")
    print("\n\n")

Some Minesweeper examples, the first one is from the lecture

In [3]:
mine_field1 = [
    ["2", "", "", "", "1"],
    ["2", "", "4", "3", ""],
    ["", "2", "", "1", ""],
    ["", "1", "", "3", ""],
    ["1", "", "", "", ""],
]
mine_field2 = [
    ["1", "", "", "", "2", "", "", "1"],
    ["", "1", "2", "", "3", "", "", "1"],
    ["", "2", "", "1", "", "", "", "0"],
    ["", "2", "1", "", "", "2", "3", "1"],
    ["", "", "", "2", "", "", "", ""],
    ["1", "", "", "", "4", "3", "", ""],
    ["", "1", "", "", "4", "", "3", ""],
    ["1", "", "", "2", "", "2", "", "1"],
]
mine_field3 = [
    ["", "", "", "", "", "1", "", ""],
    ["1", "2", "2", "1", "", "", "1", "1"],
    ["", "1", "", "", "", "", "", ""],
    ["", "", "0", "", "", "1", "", "1"],
    ["", "1", "", "", "", "", "", ""],
    ["", "", "1", "", "2", "3", "", ""],
    ["", "", "", "", "", "", "2", "1"],
    ["", "1", "", "2", "2", "", "2", ""],
]
mine_field4 = [
    ["1", "", "", "2", "", "2", "", "2", "", ""],
    ["", "3", "2", "", "", "", "4", "", "", "1"],
    ["", "", "", "1", "3", "", "", "", "4", ""],
    ["3", "", "1", "", "", "", "3", "", "", ""],
    ["", "2", "1", "", "1", "", "", "3", "", "2"],
    ["", "3", "", "2", "", "", "2", "", "1", ""],
    ["2", "", "", "3", "2", "", "", "2", "", ""],
    ["", "3", "", "", "", "3", "2", "", "", "3"],
    ["", "", "3", "", "3", "3", "", "", "", ""],
    ["", "2", "", "2", "", "", "", "", "2", ""],
]

Pick one of the examples

In [4]:
game = mine_field2

Create constraint solver

In [5]:
solver = pywrapcp.Solver("Minesweeper")

In [6]:
# Create solution_grid
solution_grid = {}

x_max = len(game)
y_max = len(game[0])

solution_grid = [[solver.IntVar(0, 1) for _x in range(x_max)] for _y in range(y_max)]
        
neighbours = [-1, 0, 1]

for x in range(x_max):
        for y in range(y_max):
            if game[x][y] not in "" and int(game[x][y]) >= 0:
                solver.Add(solution_grid[x][y] == 0)
                # this cell is the sum of all the surrounding cells
                solver.Add(
                    int(game[x][y]) == solver.Sum([solution_grid[x+ox][y+oy]
                                       for ox in neighbours for oy in neighbours
                                       if x + ox >=0 and
                                          y + oy >=0 and
                                          x + ox < x_max and
                                          y + oy < y_max
                                    ])
                )
                
            if game[x][y] not in "":
                # This cell cannot be a mine
                solver.Add(solution_grid[x][y] == 0)
                
                
print(solution_grid)

[[BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1)], [BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1)], [BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1)], [BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1)], [BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1)], [BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1), BooleanVar(0 .. 1)], [BooleanVar(0 .. 1), Boolea

Configure solver

In [7]:
all_vars = list(np.concatenate(solution_grid))

db = solver.Phase(all_vars, solver.INT_VAR_SIMPLE, solver.INT_VALUE_SIMPLE)

Start solver

In [8]:
solver.NewSearch(db)
while solver.NextSolution():
    pretty_print(solution_grid, game)

[ 1]	[  ]	[  ]	[💣]	[ 2]	[  ]	[💣]	[ 1]	

[💣]	[ 1]	[ 2]	[💣]	[ 3]	[  ]	[  ]	[ 1]	

[  ]	[ 2]	[  ]	[ 1]	[  ]	[💣]	[  ]	[ 0]	

[💣]	[ 2]	[ 1]	[  ]	[  ]	[ 2]	[ 3]	[ 1]	

[  ]	[  ]	[💣]	[ 2]	[  ]	[💣]	[  ]	[💣]	

[ 1]	[  ]	[  ]	[💣]	[ 4]	[ 3]	[💣]	[  ]	

[💣]	[ 1]	[  ]	[💣]	[ 4]	[💣]	[ 3]	[💣]	

[ 1]	[  ]	[  ]	[ 2]	[💣]	[ 2]	[  ]	[ 1]	






Cleanup

In [9]:
solver.EndSearch()

Print solver information

In [10]:
print("Solutions: {}".format(solver.Solutions()))
print("Runtime:   {}ms".format(solver.WallTime()))
print("Failures:  {}".format(solver.Failures()))
print("Branches:  {} ".format(solver.Branches()))

Solutions: 1
Runtime:   79ms
Failures:  4
Branches:  8 
